# **Why Cython ? **

One way to speed code up is to write it in Cython.
This is a optimizing static compiler witch makes it possible to acheive almost raw C like speed while in the Python Enviroment. The speed increment can be Variable: 2x, 10x, 100x,1000x depending on the use. It certainly seemed a skill to pick up.

It acheives this by static compiling, type declaring ( like in C) and using functions that are closer to the machine code and that contain less overhead.
In my case i manage to acheive a speed increment against the python version of x6. Not Bad !!!!
This could mean i can look at least one or two more moves into the future in the 8 second time given in the competition

so i transleted into functional Cython !

In [ ]:
%load_ext Cython

**- My N step lookahead minimax Agent with alfabeta pruning**
I started off by modifyng the code from the written notebook of Alexis Cook
[https://www.kaggle.com/alexisbcook/n-step-lookahead](http://)

For MiniMax Concept there is a great explanation here:
[https://www.youtube.com/watch?v=STjW3eH0Cik&list=PLUl4u3cNGP63gFHB6xb-kVBiQHYe_4hSi&index=6](http://)
this also contains a variant that takes into account the time constraints

For speed comparison purposes the agents function parts are declared above so as not to give the Cython version an unfair advantage.

In [ ]:
from learntools.core import binder
binder.bind(globals())
import random
import numpy as np

def drop_piece(grid, col, mark, config):
    next_grid = grid.copy()
    for row in range(config.rows-1, -1, -1):
        if next_grid[row][col] == 0:
            break
    next_grid[row][col] = mark
    return next_grid

# Helper function for get_heuristic: checks if window satisfies heuristic conditions
def check_window(window, num_discs, piece, config):
    return (window.count(piece) == num_discs and window.count(0) == config.inarow-num_discs)

# Helper function for get_heuristic: counts number of windows satisfying specified heuristic conditions
def count_windows(grid, num_discs, piece, config):
    num_windows = 0
    # horizontal
    for row in range(config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[row, col:col+config.inarow])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # vertical
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns):
            window = list(grid[row:row+config.inarow, col])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # positive diagonal
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    # negative diagonal
    for row in range(config.inarow-1, config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
            if check_window(window, num_discs, piece, config):
                num_windows += 1
    return num_windows


# Helper function for minimax: calculates value of heuristic for grid
def get_heuristic(grid, mark, config):
    num_threes = count_windows(grid, 3, mark, config)
    num_fours = count_windows(grid, 4, mark, config)
    num_threes_opp = count_windows(grid, 3, mark%2+1, config)
    num_fours_opp = count_windows(grid, 4, mark%2+1, config)
    score = num_threes - 1e2*num_threes_opp - 1e4*num_fours_opp + 1e6*num_fours
    return score

# Uses minimax to calculate value of dropping piece in selected column
def score_move(grid, col, mark, config, nsteps):
    next_grid = drop_piece(grid, col, mark, config)
    score = minimax(next_grid, nsteps-1, False, mark, config)
    return score

# Helper function for minimax: checks if agent or opponent has four in a row in the window
def is_terminal_window(window, config):
    return window.count(1) == config.inarow or window.count(2) == config.inarow

# Helper function for minimax: checks if game has ended
def is_terminal_node(grid, config):
    # Check for draw 
    if list(grid[0, :]).count(0) == 0:
        return True
    # Check for win: horizontal, vertical, or diagonal
    # horizontal 
    for row in range(config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[row, col:col+config.inarow])
            if is_terminal_window(window, config):
                return True
    # vertical
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns):
            window = list(grid[row:row+config.inarow, col])
            if is_terminal_window(window, config):
                return True
    # positive diagonal
    for row in range(config.rows-(config.inarow-1)):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
            if is_terminal_window(window, config):
                return True
    # negative diagonal
    for row in range(config.inarow-1, config.rows):
        for col in range(config.columns-(config.inarow-1)):
            window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
            if is_terminal_window(window, config):
                return True
    return False

# Minimax implementation with alfabeta pruning
def minimax(node, depth, maximizingPlayer, mark, config):
    is_terminal = is_terminal_node(node, config)
    valid_moves = [c for c in range(config.columns) if node[0][c] == 0]
    if is_terminal:
        return get_heuristic(node, mark, config),True
    if depth == 0 :
        return get_heuristic(node, mark, config),False

    if maximizingPlayer:
        value = -np.Inf

        for col in valid_moves:
            child = drop_piece(node, col, mark, config)
            miniv,endg =minimax(child, depth-1, False, mark, config)
            value = max(value, miniv)
            if endg==True:##ALFA BETA PRUNING
                break

        return value, False
    else:
        value = np.Inf
        for col in valid_moves:
            child = drop_piece(node, col, mark%2+1, config)
            miniv,endg = minimax(child, depth-1, True, mark, config)
            value = min(value,miniv )
            if endg==True:##ALFA BETA PRUNING
                break

        return value,False
        
def my_agent(obs, config):    
    # Your code here: Amend the agent!
    # Get list of valid moves
    valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]
    # Convert the board to a 2D grid
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    # Use the heuristic to assign a score to each possible board in the next step
    N_STEPS=3
    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config, N_STEPS) for col in valid_moves]))
    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    
    
    # Select at random from the maximizing columns 
    return random.choice(max_cols)


In [ ]:
from kaggle_environments import make, evaluate

import time

start = time.time()


# Create the game environment
env = make("connectx")


# Two random agents play one game round
moves=env.run([my_agent,'random'])#'random'

# Show the game
env.render(mode="ipython")

For the speed test i run 10 matches, and beacuse the is a randomness to how long each match is i decide to count the total of how many moves my agent decide to take in these 10 matches.
after opening up the result of env.run i came up with  [actions.append(moves[i][0]['action']) for i in range (1,len(moves),2)] to capture a list of the decisions it took.

In [ ]:
import time
def speedcheck (agent):
    start = time.time()
    actions=[];
    for i in range (3):
        moves= env.run([agent, 'random']) 
        [actions.append(moves[i][0]['action']) for i in range (1,len(moves),2)]
    end = time.time()
    print('Ran game simulations in {} s \n now have {} moves in dataset '.format((end - start),len(actions)))  
    print('{} seconds per move'.format((end - start)/len(actions)))

In [ ]:
speedcheck(my_agent)

# MY N-STEP LOOK AHEAD AGENT

In [ ]:
%%cython 
from cython.view cimport array as cvarray

cimport numpy as np
import numpy as np
import pickle 
import random

DTYPE = np.intc 


cpdef  np.ndarray[dtype= int , ndim=2] drop_piece( np.ndarray[ dtype=np.int32_t,ndim=2] grid,int col,int mark):
    ''' Gets board at next step if agent drops piece in selected column
    Purposefully creates a new board image so as not to corrupt the old one, useful behavior as same grid needs to be pass into a different
    possbile move in order for evaluation'''
    cdef np.ndarray[dtype= int , ndim=2] next_grid= grid.copy()
    cdef int [:,:] nxtgridview = next_grid
    for row in range(5, -1, -1):
        if nxtgridview[row][col] == 0:
            break
    nxtgridview[row][col] = mark
    return next_grid

# # Helper function for get_heuristic: checks if window satisfies heuristic conditions
cdef int count( int[4] window,int piece):
    cdef int count=0
    for i in range (4):          
        if window[i]== piece:
            count += 1
    return count    

cdef bint check_window( int[4] window,int num_discs,int piece ):
    return ( (count(window, piece) == num_discs) and (count(window,0) == 4-num_discs) )

# Helper function for get_heuristic: counts number of windows satisfying specified heuristic conditions

cdef int count_windows(np.ndarray[ dtype=np.int32_t,ndim=2] grid,int num_discs,int piece):
    cdef int num_windows = 0
    cdef int[4] window
    cdef int i =0
    # horizontal
    for row in range(6):
        for col in range(4):
            window = (grid[row, col:col+4]) 
            if check_window(window, num_discs, piece):
                num_windows += 1
    # vertical
    for row in range(3):
        for col in range(7):
            window = (grid[row:row+4, col]) 
            if check_window(window, num_discs, piece):
                num_windows += 1
    # positive diagonal
    for row in range(3):
        i=0
        for col in range(4):
            for i in range (4):
                window[i] = (grid[row+i,col+i])
            if check_window(window, num_discs, piece):
                num_windows += 1
    # negative diagonal
    for row in range(3, 6):
        i=0
        for col in range(4):
            for i in range (4):
                window[i] = (grid[row-i, col+i])
            if check_window(window, num_discs, piece):
                num_windows += 1
    return num_windows

    
# Helper function for minimax: calculates value of heuristic for grid
cdef int get_heuristic( np.ndarray[ dtype=np.int32_t,ndim=2] grid, int mark):
    cdef int score=0
    score = count_windows(grid, 3, mark) \
            - 100*count_windows(grid, 3, mark%2+1) \
            - 10000*count_windows(grid, 4, mark%2+1 ) \
            + 1000000*count_windows(grid, 4, mark)
    return score

# Uses minimax to calculate value of dropping piece in selected column
cdef int score_move(np.ndarray[ dtype=np.int32_t,ndim=2]grid, int col,int mark, int  nsteps):
    cdef np.ndarray[ dtype=np.int32_t,ndim=2] next_grid = drop_piece(grid, col, mark)
    score = minimax(next_grid, nsteps-1, False, mark)    
    
    return score

# Helper function for minimax: checks if agent or opponent has four in a row in the window
cdef bint is_terminal_window(int[4] window):
    return ( (count(window,1) == 4)or (count(window,2) == 4))

# Helper function for minimax: checks if game has ended
cdef bint is_terminal_node(np.ndarray[ dtype=np.int32_t,ndim=2] grid) :
    cdef int num_windows = 0
    cdef int [4] window
    cdef bint not_draw = False
    # Check for draw 
    for i in range(7):
        if grid [0, i] == 0:
            not_draw = True
    if not_draw == False:
        return True
    # Check for win: horizontal, vertical, or diagonal
    # horizontal
    for row in range(6):
        for col in range(4):
            window = (grid[row, col:col+4])
            if is_terminal_window(window):
                    return True
    # vertical
    for row in range(3):
        for col in range(7):
            window = (grid[row:row+4, col])
            if is_terminal_window(window):
                    return True
    # positive diagonal
    for row in range(3):
        for col in range(4):
#             window = (grid[range(row, row+4), range(col, col+4)])
            for i in range (4):
                window[i] = (grid[row+i,col+i])
            if is_terminal_window(window):
                return True
    # negative diagonal
    for row in range(3, 6):
        for col in range(4):
#             window = (grid[range(row, row-4, -1), range(col, col+4)])
            for i in range (4):
                window[i] = (grid[row-i, col+i])
            if is_terminal_window(window):
                return True
    return False



# Minimax implementation with alfabeta pruning
cdef int minimax(np.ndarray[ dtype=np.int32_t,ndim=2] Pnode,int depth,int maximizingPlayer,int mark):
    cdef bint is_terminal
    cdef int value
    cdef int maxv
    cdef int miniv
    cdef np.ndarray[ dtype=np.int32_t,ndim=2] node = Pnode.copy()
    cdef int [:,:] node_view = node
    cdef int col
    cdef np.ndarray[dtype= int , ndim=2] child 
    cdef int [7] val_m#valid 
    cdef int m_c=0#valid moves count
    
    for c in range(7):
        if node_view [0][c]==0 :
            val_m[m_c]=c
            m_c += 1
        
    is_terminal = is_terminal_node(node)

    if is_terminal:
        value =get_heuristic(node, mark)
        if value > 800000:
              value= 800000+ 10*(depth)  #alfabeta pruning encoding the depth of a winning path into the score. So as not 
        return value                     # bother looking for similar or longer paths.
    if depth == 0 :
        value=get_heuristic(node, mark)
        return value

    if maximizingPlayer:
        value = - 200000000 

        for c in range(m_c):
            col = val_m[c] 
            child = drop_piece(node,col, mark)  
            maxv =minimax(child, depth-1, False, mark)             
            value = max(value, maxv)
            if value >= 800000 :##ALFA BETA PRUNING
                winning_depth = (value -800000)%10
                if winning_depth >= (depth-1):
                    break  #i  Could only come up with an equally fast path. remeber depth goes down. 
        return value
    else:
        value =  200000000 
        for c in range(m_c):
            col = val_m[c]
            child = drop_piece(node,col,mark%2+1)
            miniv = minimax(child, depth-1, True, mark)
            value = min(value,miniv )
            if value < -8000:##ALFA BETA PRUNING
                break   
        return value

cpdef int my_c_agent (obs, config): 

    cdef np.ndarray[dtype= np.int32_t , ndim=2] grid = np.array(obs.board,np.intc ).reshape(6, 7)
    cdef np.ndarray[ dtype=np.int32_t,ndim=1] valid_moves  
    cdef int [7] val_m#valid 
    cdef int m_c=0#valid moves count
    cdef int counter=1 
    cdef int [:,:] gridview = grid 
    # Get list of valid moves   
    for c in range(7):
        if gridview [0][c]==0 :
            val_m[m_c]=c
            m_c += 1
#     valid_moves  = np.array([c for c in range(7) if grid[0][c] == 0],dtype= np.intc)

    # Use the heuristic to assign a score to each possible board in the next step
    N_STEPS=3

    scores = dict(zip(val_m,\
                      [score_move(grid, val_m[i], obs.mark, N_STEPS) for i in range(m_c) ] ))
    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    
    return max_cols[len(max_cols)//2]



In [ ]:
from kaggle_environments import make, evaluate

import time

start = time.time()


# Create the game environment
env = make("connectx")


# Two random agents play one game round
moves=env.run([my_c_agent,'random'])#'random'

# Show the game
env.render(mode="ipython")

# Speed Improvement, Showdown !



In [ ]:
speedcheck(my_agent)
speedcheck(my_c_agent)


That is a x6 improvement in speed !!

Great, But is it working just as fine ??  checking out my victory ratio against a random opponent. 

In [ ]:
def get_win_percentages(agent1, agent2, n_rounds=10):
    # Use default Connect Four setup
    config = {'rows': 6, 'columns': 7, 'inarow': 4}
    # Agent 1 goes first (roughly) half the time          
    outcomes = evaluate("connectx", [agent1, agent2], config, [], n_rounds//2)
    # Agent 2 goes first (roughly) half the time      
    outcomes += [[b,a] for [a,b] in evaluate("connectx", [agent2, agent1], config, [], n_rounds-n_rounds//2)]
    print("Agent 1 Win Percentage:", np.round(outcomes.count([1,-1])/len(outcomes), 2))
    print("Agent 2 Win Percentage:", np.round(outcomes.count([-1,1])/len(outcomes), 2))
    print("Number of Invalid Plays by Agent 1:", outcomes.count([None, 0]))
    print("Number of Invalid Plays by Agent 2:", outcomes.count([0, None]))


In [ ]:
get_win_percentages(agent1=my_agent, agent2="random")

Both 100 % Win Ratio,
Great, so what next ? 

Each increment of N_step should increment expenentially the computational cost. And since the base is 6 ..
our x6 gain  should just be enough to buy that extra foresight while perfoming in the same time scale as python written- my_agent
Now i can increment how Forward thinking my c Agent is by changing N -steps from 3 to 4**** *redeclaring underneath

In [ ]:
%%cython 
from cython.view cimport array as cvarray

cimport numpy as np
import numpy as np
import pickle 
import random

DTYPE = np.intc 


cpdef  np.ndarray[dtype= int , ndim=2] drop_piece( np.ndarray[ dtype=np.int32_t,ndim=2] grid,int col,int mark):
    ''' Gets board at next step if agent drops piece in selected column
    Purposefully creates a new board image so as not to corrupt the old one, useful behavior as same grid needs to be pass into a different
    possbile move in order for evaluation'''
    cdef np.ndarray[dtype= int , ndim=2] next_grid= grid.copy()
    cdef int [:,:] nxtgridview = next_grid
    for row in range(5, -1, -1):
        if nxtgridview[row][col] == 0:
            break
    nxtgridview[row][col] = mark
    return next_grid

# # Helper function for get_heuristic: checks if window satisfies heuristic conditions
cdef int count( int[4] window,int piece):
    cdef int count=0
    for i in range (4):
#         if window[i] != 0 or window[i] != 1 or window[i] != 2 :
#             filehandler = open('ERROR IN WINDOW ', 'wb') #######################################################################################
#             pickle.dump(count, filehandler)##########################################################               
        if window[i]== piece:
            count += 1
    return count    

cdef bint check_window( int[4] window,int num_discs,int piece ):
    return ( (count(window, piece) == num_discs) and (count(window,0) == 4-num_discs) )

# Helper function for get_heuristic: counts number of windows satisfying specified heuristic conditions

cdef int count_windows(np.ndarray[ dtype=np.int32_t,ndim=2] grid,int num_discs,int piece):
    cdef int num_windows = 0
    cdef int[4] window
    cdef int i =0
    # horizontal
    for row in range(6):
        for col in range(4):
            window = (grid[row, col:col+4]) 
            if check_window(window, num_discs, piece):
                num_windows += 1
    # vertical
    for row in range(3):
        for col in range(7):
            window = (grid[row:row+4, col]) 
            if check_window(window, num_discs, piece):
                num_windows += 1
    # positive diagonal
    for row in range(3):
        i=0
        for col in range(4):
            for i in range (4):
                window[i] = (grid[row+i,col+i])
            if check_window(window, num_discs, piece):
                num_windows += 1
    # negative diagonal
    for row in range(3, 6):
        i=0
        for col in range(4):
            for i in range (4):
                window[i] = (grid[row-i, col+i])
            if check_window(window, num_discs, piece):
                num_windows += 1
    return num_windows

    
# Helper function for minimax: calculates value of heuristic for grid
cdef int get_heuristic( np.ndarray[ dtype=np.int32_t,ndim=2] grid, int mark):
    cdef int score=0
#  score = num_threes - 1e2*num_threes_opp - 1e4*num_fours_opp + 1e6*num_fours
#     num_threes = count_windows(grid, 3, mark, config)
#     num_fours = count_windows(grid, 4, mark, config)
#     num_threes_opp = count_windows(grid, 3, mark%2+1, config)
#     num_fours_opp = count_windows(grid, 4, mark%2+1, config)
    score = count_windows(grid, 3, mark) \
            - 100*count_windows(grid, 3, mark%2+1) \
            - 10000*count_windows(grid, 4, mark%2+1 ) \
            + 1000000*count_windows(grid, 4, mark)
    return score

# Uses minimax to calculate value of dropping piece in selected column
cdef int score_move(np.ndarray[ dtype=np.int32_t,ndim=2]grid, int col,int mark, int  nsteps):
    cdef np.ndarray[ dtype=np.int32_t,ndim=2] next_grid = drop_piece(grid, col, mark)
    score = minimax(next_grid, nsteps-1, False, mark)    
    
    return score

# Helper function for minimax: checks if agent or opponent has four in a row in the window
cdef bint is_terminal_window(int[4] window):
    return ( (count(window,1) == 4)or (count(window,2) == 4))

# Helper function for minimax: checks if game has ended
cdef bint is_terminal_node(np.ndarray[ dtype=np.int32_t,ndim=2] grid) :
    cdef int num_windows = 0
    cdef int [4] window
    cdef bint not_draw = False
    # Check for draw 
    for i in range(7):
        if grid [0, i] == 0:
            not_draw = True
    if not_draw == False:
        return True
    # Check for win: horizontal, vertical, or diagonal
    # horizontal
    for row in range(6):
        for col in range(4):
            window = (grid[row, col:col+4])
            if is_terminal_window(window):
                    return True
    # vertical
    for row in range(3):
        for col in range(7):
            window = (grid[row:row+4, col])
            if is_terminal_window(window):
                    return True
    # positive diagonal
    for row in range(3):
        for col in range(4):
#             window = (grid[range(row, row+4), range(col, col+4)])
            for i in range (4):
                window[i] = (grid[row+i,col+i])
            if is_terminal_window(window):
                return True

    # negative diagonal
    for row in range(3, 6):
        for col in range(4):
#             window = (grid[range(row, row-4, -1), range(col, col+4)])
            for i in range (4):
                window[i] = (grid[row-i, col+i])
            if is_terminal_window(window):
                return True
    return False



# Minimax implementation with alfabeta pruning
cdef int minimax(np.ndarray[ dtype=np.int32_t,ndim=2] Pnode,int depth,int maximizingPlayer,int mark):
    cdef bint is_terminal
    cdef int value
    cdef int maxv
    cdef int miniv
    cdef np.ndarray[ dtype=np.int32_t,ndim=2] node = Pnode.copy()
    cdef int [:,:] node_view = node
    cdef int col
    cdef np.ndarray[dtype= int , ndim=2] child 
    cdef int [7] val_m#valid 
    cdef int m_c=0#valid moves count
    
    for c in range(7):
        if node_view [0][c]==0 :
            val_m[m_c]=c
            m_c += 1
        
    is_terminal = is_terminal_node(node)

    if is_terminal:
        value =get_heuristic(node, mark)
        if value > 800000:
              value= 800000+ 10*(depth)  #alfabeta pruning encoding the depth of a winning path into the score. So as not 
        return value                     # bother looking for similar or longer paths.
    if depth == 0 :
        value=get_heuristic(node, mark)
        return value

    if maximizingPlayer:
        value = - 200000000 

        for c in range(m_c):
            col = val_m[c] 
            child = drop_piece(node,col, mark)  
            maxv =minimax(child, depth-1, False, mark)             
            value = max(value, maxv)
            if value >= 800000 :##ALFA BETA PRUNING
                winning_depth = (value -800000)%10
                if winning_depth >= (depth-1):
#                     filehandler = open('AlfaBetaPruningIntervention_saving {}x{} minimax calls'.format(m_c-c,depth), 'wb') #######################################################################################
#                     pickle.dump(child, filehandler)##########################################################    
                    break  #i  Could only come up with an equally fast path. remeber depth goes down. 
        return value
    else:
        value =  200000000 
        for c in range(m_c):
            col = val_m[c]
            child = drop_piece(node,col,mark%2+1)
            miniv = minimax(child, depth-1, True, mark)
            value = min(value,miniv )
            if value < -8000:##ALFA BETA PRUNING
                break   
        return value

cpdef int my_c_agent4 (obs, config): 

    cdef np.ndarray[dtype= np.int32_t , ndim=2] grid = np.array(obs.board,np.intc ).reshape(6, 7)
    cdef np.ndarray[ dtype=np.int32_t,ndim=1] valid_moves  
    # Your code here: Amend the agent!
    # Get list of valid moves
    cdef int [7] val_m#valid 
    cdef int m_c=0#valid moves count
    cdef int counter=1 #######################################################
    cdef int [:,:] gridview = grid ##############################################
 

    # Convert the board to a 2D grid
    
    
    for c in range(7):
        if gridview [0][c]==0 :
            val_m[m_c]=c
            m_c += 1
#     valid_moves  = np.array([c for c in range(7) if grid[0][c] == 0],dtype= np.intc)

    # Use the heuristic to assign a score to each possible board in the next step
    N_STEPS=4

    scores = dict(zip(val_m,\
                      [score_move(grid, val_m[i], obs.mark, N_STEPS) for i in range(m_c) ] ))
    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    

#     for r in range(6):#################################
#         for c in range(7):
#             if gridview[r,c] != 0 :
#                 counter += 1
#                 
#     filehandler = open('MyAgents_scores_move{}'.format(counter), 'wb') #######################################################################################
#     pickle.dump(scores, filehandler)##########################################################      


    # Select at random from the maximizing columns 
#       return random.choice(max_cols)
    # slightly improved the version by eliminating the randomness with a bit of strategy - conquering the center of the board
    return max_cols[len(max_cols)//2]

In [ ]:
speedcheck(my_c_agent4)

NOW I CAN PLAY THEM AGAINST EACH OTHER !

In [ ]:
from kaggle_environments import make, evaluate

import time

start = time.time()


# Create the game environment
env = make("connectx")


# Two random agents play one game round
moves=env.run([my_c_agent4, my_c_agent])

# Show the game
env.render(mode="ipython")

**Draw !!**

Turns out that in this game with this size grid with the player thinking equally there is not much advantage of being able too see 4 moves ahead rather than 3. - Especially if we play conservatively !!

But seeing one move ahead does make us able to play less defensivly. i can decrease the penalty of the adversary getting 3 in a row to be equal but contrary as to my getting 3 Change in (get_heuristics) . I can do this because with the actual extra 1 step sight i can immagine him getting 4 and scoring the big penalty- if it is an actual risk wich I can better assert.

In [ ]:
%%cython 
from cython.view cimport array as cvarray

cimport numpy as np
import numpy as np
import pickle 
import random

DTYPE = np.intc 


cpdef  np.ndarray[dtype= int , ndim=2] drop_piece( np.ndarray[ dtype=np.int32_t,ndim=2] grid,int col,int mark):
    ''' Gets board at next step if agent drops piece in selected column
    Purposefully creates a new board image so as not to corrupt the old one, useful behavior as same grid needs to be pass into a different
    possbile move in order for evaluation'''
    cdef np.ndarray[dtype= int , ndim=2] next_grid= grid.copy()
    cdef int [:,:] nxtgridview = next_grid
    for row in range(5, -1, -1):
        if nxtgridview[row][col] == 0:
            break
    nxtgridview[row][col] = mark
    return next_grid

# # Helper function for get_heuristic: checks if window satisfies heuristic conditions
cdef int count( int[4] window,int piece):
    cdef int count=0
    for i in range (4):
#         if window[i] != 0 or window[i] != 1 or window[i] != 2 :
#             filehandler = open('ERROR IN WINDOW ', 'wb') #######################################################################################
#             pickle.dump(count, filehandler)##########################################################               
        if window[i]== piece:
            count += 1
    return count    

cdef bint check_window( int[4] window,int num_discs,int piece ):
    return ( (count(window, piece) == num_discs) and (count(window,0) == 4-num_discs) )

# Helper function for get_heuristic: counts number of windows satisfying specified heuristic conditions

cdef int count_windows(np.ndarray[ dtype=np.int32_t,ndim=2] grid,int num_discs,int piece):
    cdef int num_windows = 0
    cdef int[4] window
    cdef int i =0
    # horizontal
    for row in range(6):
        for col in range(4):
            window = (grid[row, col:col+4]) 
            if check_window(window, num_discs, piece):
                num_windows += 1
    # vertical
    for row in range(3):
        for col in range(7):
            window = (grid[row:row+4, col]) 
            if check_window(window, num_discs, piece):
                num_windows += 1
    # positive diagonal
    for row in range(3):
        i=0
        for col in range(4):
            for i in range (4):
                window[i] = (grid[row+i,col+i])
            if check_window(window, num_discs, piece):
                num_windows += 1
    # negative diagonal
    for row in range(3, 6):
        i=0
        for col in range(4):
            for i in range (4):
                window[i] = (grid[row-i, col+i])
            if check_window(window, num_discs, piece):
                num_windows += 1
    return num_windows

    
# Helper function for minimax: calculates value of heuristic for grid
cdef int get_heuristic( np.ndarray[ dtype=np.int32_t,ndim=2] grid, int mark):
    cdef int score=0
    score = count_windows(grid, 3, mark) \
            - 1*count_windows(grid, 3, mark%2+1) \
            - 10000*count_windows(grid, 4, mark%2+1 ) \
            + 1000000*count_windows(grid, 4, mark)
    return score

# Uses minimax to calculate value of dropping piece in selected column
cdef int score_move(np.ndarray[ dtype=np.int32_t,ndim=2]grid, int col,int mark, int  nsteps):
    cdef np.ndarray[ dtype=np.int32_t,ndim=2] next_grid = drop_piece(grid, col, mark)
    score = minimax(next_grid, nsteps-1, False, mark)    
    
    return score

# Helper function for minimax: checks if agent or opponent has four in a row in the window
cdef bint is_terminal_window(int[4] window):
    return ( (count(window,1) == 4)or (count(window,2) == 4))

# Helper function for minimax: checks if game has ended
cdef bint is_terminal_node(np.ndarray[ dtype=np.int32_t,ndim=2] grid) :
    cdef int num_windows = 0
    cdef int [4] window
    cdef bint not_draw = False
    # Check for draw 
    for i in range(7):
        if grid [0, i] == 0:
            not_draw = True
    if not_draw == False:
        return True
    # Check for win: horizontal, vertical, or diagonal
    # horizontal
    for row in range(6):
        for col in range(4):
            window = (grid[row, col:col+4])
            if is_terminal_window(window):
                    return True
    # vertical
    for row in range(3):
        for col in range(7):
            window = (grid[row:row+4, col])
            if is_terminal_window(window):
                    return True
    # positive diagonal
    for row in range(3):
        for col in range(4):
#             window = (grid[range(row, row+4), range(col, col+4)])
            for i in range (4):
                window[i] = (grid[row+i,col+i])
            if is_terminal_window(window):
                return True

    # negative diagonal
    for row in range(3, 6):
        for col in range(4):
#             window = (grid[range(row, row-4, -1), range(col, col+4)])
            for i in range (4):
                window[i] = (grid[row-i, col+i])
            if is_terminal_window(window):
                return True
    return False



# Minimax implementation with alfabeta pruning
cdef int minimax(np.ndarray[ dtype=np.int32_t,ndim=2] Pnode,int depth,int maximizingPlayer,int mark):
    cdef bint is_terminal
    cdef int value
    cdef int maxv
    cdef int miniv
    cdef np.ndarray[ dtype=np.int32_t,ndim=2] node = Pnode.copy()
    cdef int [:,:] node_view = node
    cdef int col
    cdef np.ndarray[dtype= int , ndim=2] child 
    cdef int [7] val_m#valid 
    cdef int m_c=0#valid moves count
    # Create valid moves list
    for c in range(7):
        if node_view [0][c]==0 :
            val_m[m_c]=c
            m_c += 1
        
    is_terminal = is_terminal_node(node)
    if is_terminal:
        value =get_heuristic(node, mark)
        if value > 800000:
              value= 800000+ 10*(depth)  #alfabeta pruning encoding the depth of a winning path into the score. So as not 
        return value                     # bother looking for similar or longer paths.
    if depth == 0 :
        value=get_heuristic(node, mark)
        return value

    if maximizingPlayer:
        value = - 200000000 
        for c in range(m_c):
            col = val_m[c] 
            child = drop_piece(node,col, mark)  
            maxv =minimax(child, depth-1, False, mark)             
            value = max(value, maxv)
            if value >= 800000 :##ALFA BETA PRUNING
                winning_depth = (value -800000)%10
                if winning_depth >= (depth-1):
                    break  #i  Could only come up with an equally fast path. remeber depth goes down. 
        return value
    else:
        value =  200000000 
        for c in range(m_c):
            col = val_m[c]
            child = drop_piece(node,col,mark%2+1)
            miniv = minimax(child, depth-1, True, mark)
            value = min(value,miniv )
            if value < -8000:##ALFA BETA PRUNING
                break   
        return value

cpdef int my_c_agent4a (obs, config): 

    cdef np.ndarray[dtype= np.int32_t , ndim=2] grid = np.array(obs.board,np.intc ).reshape(6, 7)
    cdef np.ndarray[ dtype=np.int32_t,ndim=1] valid_moves  
    cdef int [7] val_m#valid 
    cdef int m_c=0#valid moves count
    cdef int counter=1 
    cdef int [:,:] gridview = grid    
    # Create valid moves list
    for c in range(7):
        if gridview [0][c]==0 :
            val_m[m_c]=c
            m_c += 1

    # Use the heuristic to assign a score to each possible board in the next step
    if m_c >= 6:
        N_STEPS=4
    if m_c ==5:
        N_STEPS=4
    if m_c ==4:
        N_STEPS=5
    if m_c <=3:
        N_STEPS=7

        
    scores = dict(zip(val_m,[score_move(grid, val_m[i], obs.mark, N_STEPS) for i in range(m_c) ] ))
    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    return max_cols[len(max_cols)//2]

Another thought was to increase N_steps when the possible moves are less, Still exponential but with a smaller base.

In [ ]:
speedcheck(my_c_agent4a)

In [ ]:
get_win_percentages(agent1=my_agent, agent2=my_c_agent4a)

Opponent Stood no chance !!  and I still give an answer in the same time period !!**

# Final thoughts

THIS was just a pipline part of a bigger project awithc was to make my first RL machine that could initially learn of from the data that i generate with my_c_agent. Training NN takes up a lot of Data !! ( i was aiming at 50.000 Move dataset for my 5.000 Neuron Neural Net. so you see why speed was important to me !! 

See the rest of the project here: link in comment down below !

If you liked it, want to know more about the fondamentals of Cython please Upvote and write in the comments Every tip i learnt i am happy to share ! As you can see there are a lot of debugging point i left in the code, in my_c_agent4 ( the hidden one). There we a lot of problems encountered and lessons learned. Gonna drop Some in with the Tricks in an upcomming notebook !! 
-link in comments

**Ciao and thanks for reading !**

# Play against my Best Agent Here !

In [ ]:
from kaggle_environments import evaluate, make, utils

env = make("connectx", debug=True)
env.play([my_c_agent4a, None], width=500, height=450)